# Understanding Python's asyncio

This notebook provides a comprehensive tutorial on Python's `asyncio` library, designed to teach you asynchronous programming concepts from the ground up.

## Contents

1. [Introduction to Asynchronous Programming](#1-introduction-to-asynchronous-programming)
2. [Asyncio Basics](#2-asyncio-basics)
3. [Working with Tasks and Futures](#3-working-with-tasks-and-futures)
4. [Advanced Asyncio Concepts](#4-advanced-asyncio-concepts)
5. [Practical Examples](#5-practical-examples)
6. [Best Practices and Patterns](#6-best-practices-and-patterns)

## 1. Introduction to Asynchronous Programming

### What is Asynchronous Programming?

Asynchronous programming is a programming paradigm that allows multiple operations to be executed concurrently (not necessarily in parallel) without blocking the main execution thread. In contrast to synchronous programming where operations are executed sequentially, asynchronous programming enables non-blocking execution, making it ideal for I/O-bound operations.

### Synchronous vs. Asynchronous: A Practical Example

Let's compare synchronous and asynchronous approaches to understand the difference:

In [ ]:
import time

# Synchronous function to simulate an I/O operation
def sync_io_operation(operation_name, duration):
    print(f"Starting {operation_name}...")
    time.sleep(duration)  # Blocking operation
    print(f"Finished {operation_name} after {duration}s")
    return f"{operation_name} result"

# Simulating multiple synchronous operations
def run_sync_operations():
    start = time.time()
    
    result1 = sync_io_operation("Operation 1", 2)
    result2 = sync_io_operation("Operation 2", 1)
    result3 = sync_io_operation("Operation 3", 3)
    
    end = time.time()
    print(f"Total synchronous execution time: {end - start:.2f}s")
    return [result1, result2, result3]

# Run the synchronous operations
run_sync_operations()

Now, let's look at the same operations executed asynchronously using `asyncio`:

In [ ]:
import asyncio

# Asynchronous function to simulate an I/O operation
async def async_io_operation(operation_name, duration):
    print(f"Starting {operation_name}...")
    await asyncio.sleep(duration)  # Non-blocking operation
    print(f"Finished {operation_name} after {duration}s")
    return f"{operation_name} result"

# Simulating multiple asynchronous operations
async def run_async_operations():
    start = time.time()
    
    # Create tasks to run concurrently
    task1 = asyncio.create_task(async_io_operation("Operation 1", 2))
    task2 = asyncio.create_task(async_io_operation("Operation 2", 1))
    task3 = asyncio.create_task(async_io_operation("Operation 3", 3))
    
    # Wait for all tasks to complete
    results = await asyncio.gather(task1, task2, task3)
    
    end = time.time()
    print(f"Total asynchronous execution time: {end - start:.2f}s")
    return results

# Run the asynchronous operations
await run_async_operations()

### When to Use Asyncio

Asyncio is particularly useful for:
- I/O-bound operations (network requests, file operations)
- Handling many concurrent connections (web servers, chat applications)
- Operations that involve waiting (APIs with rate limits, scheduled tasks)

It's less suitable for:
- CPU-bound tasks (use multiprocessing instead)
- Simple sequential operations with no waiting
- Small scripts with minimal I/O operations

## 2. Asyncio Basics

### Coroutines and the `async`/`await` Syntax

A coroutine is a specialized version of a Python generator function that can suspend and resume execution. They're created using the `async def` syntax.

In [ ]:
# Define a coroutine
async def my_coroutine():
    print("Coroutine started")
    await asyncio.sleep(1)  # Give up control and allow other coroutines to run
    print("Coroutine resumed after 1 second")
    return "Coroutine completed"

# This won't execute the coroutine, it just creates a coroutine object
coro = my_coroutine()
print(f"Type of my_coroutine(): {type(coro)}")

# To actually run a coroutine, you need to schedule it on the event loop
result = await my_coroutine()
print(f"Result: {result}")

### Understanding the Event Loop

The event loop is the core of every asyncio application. It's responsible for:
- Scheduling and running asyncio tasks
- Handling I/O events
- Running subprocesses
- Managing timeouts

In [ ]:
# Get the current event loop
loop = asyncio.get_event_loop()

# In Jupyter notebooks, the event loop is already running (in IPython)
# In regular Python scripts, you would manually run the loop like this:
'''
# For Python 3.7+
asyncio.run(my_coroutine())  # Creates a new event loop and runs the coroutine

# For older Python versions
loop = asyncio.get_event_loop()
result = loop.run_until_complete(my_coroutine())
'''

### Running Multiple Coroutines

There are several ways to run multiple coroutines:

In [ ]:
async def task1():
    await asyncio.sleep(1)
    print("Task 1 completed")
    return "Result 1"

async def task2():
    await asyncio.sleep(0.5)
    print("Task 2 completed")
    return "Result 2"

# Method 1: Using asyncio.gather()
print("Running tasks with asyncio.gather()")
results = await asyncio.gather(task1(), task2())
print(f"Results: {results}")

# Method 2: Creating and awaiting Tasks
print("\nRunning tasks with asyncio.create_task()")
t1 = asyncio.create_task(task1())
t2 = asyncio.create_task(task2())
await t1
await t2
print(f"Results: {t1.result()}, {t2.result()}")

## 3. Working with Tasks and Futures

### Tasks

A Task is a wrapper around a coroutine that schedules it to run on the event loop. Tasks allow you to run coroutines concurrently.

In [ ]:
async def long_running_task(name, duration):
    print(f"{name} started")
    await asyncio.sleep(duration)
    print(f"{name} completed after {duration}s")
    return f"{name} result"

async def manage_tasks():
    # Create tasks
    task_a = asyncio.create_task(long_running_task("Task A", 3))
    task_b = asyncio.create_task(long_running_task("Task B", 2))
    task_c = asyncio.create_task(long_running_task("Task C", 1))
    
    # Wait for all tasks to complete
    await asyncio.gather(task_a, task_b, task_c)
    
    print(f"Task A result: {task_a.result()}")
    print(f"Task B result: {task_b.result()}")
    print(f"Task C result: {task_c.result()}")

await manage_tasks()

### Task Cancellation

You can cancel running tasks:

In [ ]:
async def cancellable_task():
    try:
        print("Task started")
        while True:  # Infinite loop
            print("Working...")
            await asyncio.sleep(0.5)
    except asyncio.CancelledError:
        print("Task was cancelled!")
        raise  # Re-raise to properly handle cancellation

async def cancel_after_delay(task, delay):
    await asyncio.sleep(delay)
    task.cancel()
    print(f"Cancellation request sent after {delay}s")

async def demo_cancellation():
    task = asyncio.create_task(cancellable_task())
    # Schedule the task to be cancelled after 2 seconds
    asyncio.create_task(cancel_after_delay(task, 2))
    
    try:
        await task
    except asyncio.CancelledError:
        print("Main function caught the cancellation")

await demo_cancellation()

### Futures

A Future is a low-level awaitable object that represents an eventual result of an asynchronous operation. Tasks are a subclass of Future.

In [ ]:
async def set_future_result(future, value, delay):
    await asyncio.sleep(delay)
    future.set_result(value)

async def future_demo():
    # Create a future
    future = asyncio.Future()
    
    # Schedule a coroutine to set the future's result
    asyncio.create_task(set_future_result(future, "Future result", 2))
    
    # Wait for the future to have a result
    print("Waiting for future result...")
    result = await future
    print(f"Got result: {result}")

await future_demo()

### Waiting for Multiple Tasks

You can wait for multiple tasks with different strategies:

In [ ]:
async def wait_demo():
    tasks = [
        asyncio.create_task(long_running_task("Task X", 3)),
        asyncio.create_task(long_running_task("Task Y", 1)),
        asyncio.create_task(long_running_task("Task Z", 2))
    ]
    
    # Wait for the first task to complete
    print("\nWaiting for the first task to complete...")
    done, pending = await asyncio.wait(tasks, return_when=asyncio.FIRST_COMPLETED)
    
    print(f"\n{len(done)} task(s) completed:")
    for task in done:
        print(f"  - {task.result()}")
        
    print(f"\n{len(pending)} task(s) still pending")
    
    # Wait for the remaining tasks
    print("\nWaiting for remaining tasks...")
    done, pending = await asyncio.wait(pending)
    
    print(f"\nAll tasks completed. Results:")
    for task in tasks:
        print(f"  - {task.result()}")

await wait_demo()

## 4. Advanced Asyncio Concepts

### Timeouts

You can add timeouts to operations to prevent them from running too long:

In [ ]:
async def potentially_slow_operation():
    print("Slow operation started")
    await asyncio.sleep(5)  # Simulate a slow operation
    print("Slow operation finished")
    return "Operation result"

async def timeout_demo():
    try:
        # Set a 2-second timeout for the operation
        result = await asyncio.wait_for(potentially_slow_operation(), timeout=2)
        print(f"Got result: {result}")
    except asyncio.TimeoutError:
        print("Operation timed out!")

await timeout_demo()

### Asyncio Queues

Asyncio provides queue implementations for coordinating between producer and consumer coroutines:

In [ ]:
async def producer(queue, name, items):
    for i in range(items):
        item = f"{name} item {i}"
        await queue.put(item)
        print(f"{name} produced: {item}")
        await asyncio.sleep(0.5)
    print(f"{name} finished producing")

async def consumer(queue, name, items_to_process):
    for _ in range(items_to_process):
        item = await queue.get()
        print(f"{name} consumed: {item}")
        queue.task_done()
        await asyncio.sleep(1)  # Simulate processing time
    print(f"{name} finished consuming")

async def queue_demo():
    # Create a queue
    queue = asyncio.Queue(maxsize=5)  # Limit queue size to 5 items
    
    # Create tasks for producers and consumers
    producer_tasks = [
        asyncio.create_task(producer(queue, "Producer 1", 4)),
        asyncio.create_task(producer(queue, "Producer 2", 3))
    ]
    
    consumer_tasks = [
        asyncio.create_task(consumer(queue, "Consumer 1", 3)),
        asyncio.create_task(consumer(queue, "Consumer 2", 4))
    ]
    
    # Wait for all producers to finish
    await asyncio.gather(*producer_tasks)
    
    # Wait for the queue to be fully processed
    await queue.join()
    
    # Wait for all consumers to finish
    await asyncio.gather(*consumer_tasks)
    
    print("Queue demo completed")

await queue_demo()

### Synchronization Primitives

Asyncio provides synchronization primitives similar to the ones in the `threading` module:

In [ ]:
async def worker(lock, worker_id, shared_resource):
    print(f"Worker {worker_id} is waiting for the lock")
    async with lock:  # Acquire and release the lock automatically
        print(f"Worker {worker_id} acquired the lock")
        # Simulate working with a shared resource
        shared_resource.append(worker_id)
        print(f"Worker {worker_id} updated shared resource: {shared_resource}")
        await asyncio.sleep(1)  # Simulate some work
    print(f"Worker {worker_id} released the lock")

async def lock_demo():
    # Create a lock
    lock = asyncio.Lock()
    shared_resource = []
    
    # Create and run multiple workers concurrently
    workers = [worker(lock, i, shared_resource) for i in range(3)]
    await asyncio.gather(*workers)
    
    print(f"Final shared resource: {shared_resource}")

await lock_demo()

## 5. Practical Examples

### Asynchronous Web Requests

Let's use `aiohttp` to make concurrent web requests (install with `pip install aiohttp` if needed):

In [ ]:
try:
    import aiohttp
except ImportError:
    print("aiohttp is not installed. Please install it with:")
    print("pip install aiohttp")
    print("\nFor now, we'll skip this example.")
else:
    async def fetch_url(session, url):
        print(f"Fetching {url}")
        async with session.get(url) as response:
            if response.status != 200:
                return f"Error: {response.status} for {url}"
            data = await response.text()
            return f"Fetched {len(data)} bytes from {url}"

    async def fetch_multiple_urls():
        urls = [
            "https://example.com",
            "https://python.org",
            "https://docs.python.org/"
        ]
        
        async with aiohttp.ClientSession() as session:
            tasks = [fetch_url(session, url) for url in urls]
            results = await asyncio.gather(*tasks)
            
            for i, result in enumerate(results):
                print(f"Result {i+1}: {result}")

    # Run the example if aiohttp is installed
    try:
        await fetch_multiple_urls()
    except NameError:
        pass  # Skip if aiohttp isn't installed

### File I/O with aiofiles

Let's use `aiofiles` for asynchronous file operations (install with `pip install aiofiles` if needed):

In [ ]:
try:
    import aiofiles
except ImportError:
    print("aiofiles is not installed. Please install it with:")
    print("pip install aiofiles")
    print("\nFor now, we'll skip this example.")
else:
    async def write_file(filename, content):
        print(f"Writing to {filename}")
        async with aiofiles.open(filename, 'w') as file:
            await file.write(content)
        print(f"Finished writing to {filename}")

    async def read_file(filename):
        print(f"Reading from {filename}")
        async with aiofiles.open(filename, 'r') as file:
            content = await file.read()
        print(f"Read {len(content)} bytes from {filename}")
        return content

    async def file_io_demo():
        # Write files concurrently
        write_tasks = [
            write_file('asyncio_demo_1.txt', 'This is file 1 content\n' * 1000),
            write_file('asyncio_demo_2.txt', 'This is file 2 content\n' * 1000),
            write_file('asyncio_demo_3.txt', 'This is file 3 content\n' * 1000)
        ]
        await asyncio.gather(*write_tasks)
        
        # Read files concurrently
        read_tasks = [
            read_file('asyncio_demo_1.txt'),
            read_file('asyncio_demo_2.txt'),
            read_file('asyncio_demo_3.txt')
        ]
        results = await asyncio.gather(*read_tasks)
        
        # Clean up demo files (optional)
        import os
        for i in range(1, 4):
            os.remove(f'asyncio_demo_{i}.txt')
            
        print("File I/O demo completed")

    # Run the example if aiofiles is installed
    try:
        await file_io_demo()
    except NameError:
        pass  # Skip if aiofiles isn't installed

### Building a Simple Asynchronous Server

Let's build a very simple echo server using asyncio streams:

In [ ]:
async def handle_client(reader, writer):
    # Get client address
    addr = writer.get_extra_info('peername')
    print(f"Client connected: {addr}")
    
    while True:
        # Read data from the client
        data = await reader.read(100)
        message = data.decode()
        
        if not data:  # Client disconnected
            break
            
        print(f"Received from {addr}: {message.strip()}")
        
        # Echo the message back to the client
        response = f"Echo: {message}"
        writer.write(response.encode())
        await writer.drain()
        
    # Close the connection
    print(f"Client disconnected: {addr}")
    writer.close()
    await writer.wait_closed()

async def run_server():
    server = await asyncio.start_server(
        handle_client, '127.0.0.1', 8888)
    
    # Get server address
    addr = server.sockets[0].getsockname()
    print(f"Server running on {addr}")
    
    print("To test the server:")
    print("1. Open another terminal")
    print("2. Use: telnet 127.0.0.1 8888")
    print("3. Type messages and see them echoed back")
    print("4. Press Ctrl+C here to stop the server")
    
    try:
        async with server:
            # This will keep the server running until cancelled
            await server.serve_forever()
    except asyncio.CancelledError:
        print("Server stopped")

# Uncomment to run the server (note: this will block the notebook until cancelled)
# await run_server()

### Simulating a Real-world Application: Web Crawler

Let's build a simple asynchronous web crawler that respects rate limits:

In [ ]:
try:
    import aiohttp
    from bs4 import BeautifulSoup
except ImportError:
    print("Required packages not installed. Please install them with:")
    print("pip install aiohttp beautifulsoup4")
    print("\nSkipping this example.")
else:
    class AsyncCrawler:
        def __init__(self, base_url, max_urls=10, rate_limit=1):
            self.base_url = base_url
            self.to_visit = asyncio.Queue()
            self.visited = set()
            self.max_urls = max_urls
            self.rate_limit = rate_limit  # Seconds between requests
            self.session = None
            self.rate_limiter = asyncio.Semaphore(1)  # Only 1 request at a time
            
        async def __aenter__(self):
            self.session = aiohttp.ClientSession()
            await self.to_visit.put(self.base_url)
            return self
            
        async def __aexit__(self, exc_type, exc_val, exc_tb):
            await self.session.close()
            
        async def fetch_url(self, url):
            async with self.rate_limiter:
                print(f"Fetching: {url}")
                try:
                    async with self.session.get(url) as response:
                        if response.status != 200:
                            print(f"Error: {response.status} for {url}")
                            return None
                            
                        html = await response.text()
                        print(f"Fetched {len(html)} bytes from {url}")
                        # Respect rate limit
                        await asyncio.sleep(self.rate_limit)
                        return html
                except Exception as e:
                    print(f"Error fetching {url}: {e}")
                    return None
                    
        def extract_links(self, html, url):
            soup = BeautifulSoup(html, 'html.parser')
            links = set()
            
            for link in soup.find_all('a', href=True):
                href = link['href']
                
                # Handle relative URLs
                if href.startswith('/'):
                    href = f"{self.base_url.rstrip('/')}{href}"
                    
                # Only follow links from the same domain
                if href.startswith(self.base_url) and href not in self.visited:
                    links.add(href)
                    
            print(f"Found {len(links)} new links on {url}")
            return links
            
        async def crawl(self):
            while len(self.visited) < self.max_urls:
                if self.to_visit.empty():
                    print("No more URLs to visit")
                    break
                    
                url = await self.to_visit.get()
                
                if url in self.visited:
                    continue
                    
                self.visited.add(url)
                html = await self.fetch_url(url)
                
                if html:
                    links = self.extract_links(html, url)
                    for link in links:
                        if len(self.visited) + self.to_visit.qsize() < self.max_urls:
                            await self.to_visit.put(link)
                            
            print(f"Crawling complete. Visited {len(self.visited)} URLs")
            return self.visited

    async def run_crawler_demo():
        print("Starting web crawler demo")
        async with AsyncCrawler("https://docs.python.org/", max_urls=5, rate_limit=1) as crawler:
            visited_urls = await crawler.crawl()
            print("\nVisited URLs:")
            for i, url in enumerate(visited_urls, 1):
                print(f"{i}. {url}")

    # Run the crawler example if required packages are installed
    try:
        # Uncomment to run the crawler demo
        # await run_crawler_demo()
        pass
    except NameError:
        pass  # Skip if required packages aren't installed

## 6. Best Practices and Patterns

### Error Handling in Asyncio

Proper error handling is crucial in asynchronous code:

In [ ]:
async def might_fail(success_rate=0.5):
    import random
    await asyncio.sleep(1)  # Simulate some work
    if random.random() > success_rate:
        raise ValueError("Operation failed")
    return "Operation succeeded"

async def error_handling_demo():
    # Method 1: Try/except around individual tasks
    print("Method 1: Individual try/except:")
    for i in range(3):
        try:
            result = await might_fail(0.3)
            print(f"Task {i}: {result}")
        except ValueError as e:
            print(f"Task {i} failed: {e}")
    
    # Method 2: gather() with return_exceptions=True
    print("\nMethod 2: gather with return_exceptions=True:")
    tasks = [might_fail(0.3) for _ in range(3)]
    results = await asyncio.gather(*tasks, return_exceptions=True)
    
    for i, result in enumerate(results):
        if isinstance(result, Exception):
            print(f"Task {i} failed: {result}")
        else:
            print(f"Task {i}: {result}")
    
    # Method 3: Creating tasks and handling exceptions with callbacks
    print("\nMethod 3: Task callbacks:")
    
    def handle_task_result(task):
        try:
            result = task.result()
            print(f"Task completed: {result}")
        except Exception as e:
            print(f"Task failed: {e}")
    
    tasks = []
    for i in range(3):
        task = asyncio.create_task(might_fail(0.3))
        task.add_done_callback(handle_task_result)
        tasks.append(task)
    
    # Wait for all tasks to complete
    await asyncio.gather(*tasks, return_exceptions=True)

await error_handling_demo()

### Debugging Asyncio Code

Asyncio provides debugging options to help track down issues:

In [ ]:
# Enable debug mode
def debug_demo():
    """This is just informational - you would run this in a Python script, not in a notebook"""
    # In a Python script, you can enable debug mode:
    """
    import asyncio
    
    # Method 1: Environment variable
    # Set PYTHONASYNCIODEBUG=1 before running your script
    
    # Method 2: Using the event loop's debug mode
    asyncio.get_event_loop().set_debug(True)
    
    # Method 3: In Python 3.7+
    # asyncio.run(main(), debug=True)
    """
    
    # Debug features include:
    # - More detailed logging
    # - Warning about slow callbacks (>100ms)
    # - Resource tracking (detect unclosed resources)
    # - Exception handling improvements (more details about where exceptions occurred)

# Tips for effective asyncio debugging:
async def debugging_tips():
    """This is just for documentation - these are strategies you would apply in real code"""
    # 1. Use meaningful task names
    task = asyncio.create_task(long_running_task("Task A", 3), name="fetch_user_data")
    
    # 2. Use logging instead of print
    import logging
    logging.basicConfig(level=logging.DEBUG)
    logging.debug("Task started")
    
    # 3. Run with debug mode in scripts
    # asyncio.run(main(), debug=True)
    
    # 4. Use asyncio.current_task() to get the current task
    current = asyncio.current_task()
    print(f"Current task: {current.get_name()}")
    
    # 5. Use asyncio.all_tasks() to see all tasks
    all_tasks = asyncio.all_tasks()
    print(f"Total tasks: {len(all_tasks)}")

### Common Asyncio Patterns

Here are some common patterns you'll see with asyncio:

In [ ]:
# Pattern 1: Scatter-gather (process multiple items in parallel)
async def process_item(item):
    await asyncio.sleep(1)  # Simulate processing
    return f"Processed {item}"

async def scatter_gather_pattern(items):
    tasks = [process_item(item) for item in items]
    results = await asyncio.gather(*tasks)
    return results

# Pattern 2: Producer-Consumer with a queue
async def producer_consumer_pattern(num_producers, num_consumers, num_items):
    queue = asyncio.Queue()
    
    # Producer function
    async def producer(producer_id, n_items):
        for i in range(n_items):
            item = f"Producer {producer_id} - Item {i}"
            await queue.put(item)
            await asyncio.sleep(0.1)  # Simulate production time
        await queue.put(None)  # Sentinel to signal completion
    
    # Consumer function
    async def consumer(consumer_id):
        while True:
            item = await queue.get()
            if item is None:  # Check for sentinel
                await queue.put(None)  # Put sentinel back for other consumers
                break
            print(f"Consumer {consumer_id} processing {item}")
            await asyncio.sleep(0.5)  # Simulate consumption time
            queue.task_done()
    
    # Create and start producers
    producer_tasks = [asyncio.create_task(producer(i, num_items // num_producers)) 
                      for i in range(num_producers)]
    
    # Create and start consumers
    consumer_tasks = [asyncio.create_task(consumer(i)) 
                     for i in range(num_consumers)]
    
    # Wait for all producers to finish
    await asyncio.gather(*producer_tasks)
    
    # Wait for all consumers to finish
    await asyncio.gather(*consumer_tasks)

# Pattern 3: Throttled API calls with semaphores
async def api_call(url, semaphore):
    async with semaphore:  # Limit the number of concurrent API calls
        print(f"Calling API: {url}")
        await asyncio.sleep(1)  # Simulate API call
        return f"Result from {url}"

async def throttled_api_pattern(urls, max_concurrent=3):
    semaphore = asyncio.Semaphore(max_concurrent)
    tasks = [api_call(url, semaphore) for url in urls]
    results = await asyncio.gather(*tasks)
    return results

# Demonstrate a few of these patterns
async def demonstrate_patterns():
    print("Pattern 1: Scatter-Gather")
    items = ['item1', 'item2', 'item3', 'item4']
    results = await scatter_gather_pattern(items)
    print(f"Results: {results}")
    
    print("\nPattern 3: Throttled API Calls")
    urls = ['https://api.example.com/' + str(i) for i in range(1, 6)]
    results = await throttled_api_pattern(urls, max_concurrent=2)
    print(f"Results: {results}")
    
    # Pattern 2 is more verbose, so let's run a small version
    print("\nPattern 2: Producer-Consumer")
    await producer_consumer_pattern(num_producers=2, num_consumers=2, num_items=4)

await demonstrate_patterns()

### Common Pitfalls to Avoid

Here are some common mistakes when working with asyncio:

In [ ]:
# Pitfall 1: Forgetting to await a coroutine
async def pitfall_not_awaiting():
    async def my_coro():
        await asyncio.sleep(1)
        print("Coroutine executed")
        return "Result"
    
    print("\nPitfall 1: Forgetting to await a coroutine")
    # Wrong: This doesn't execute the coroutine, just creates a coroutine object
    my_coro()  # This will show a warning in Python 3.8+
    
    # Correct: This executes the coroutine
    await my_coro()

# Pitfall 2: Blocking the event loop with CPU-bound or blocking operations
async def pitfall_blocking_loop():
    print("\nPitfall 2: Blocking the event loop")
    
    # Wrong: This blocks the event loop, preventing other coroutines from running
    def heavy_computation():
        print("Starting heavy computation...")
        # Simulate a CPU-intensive task
        result = 0
        for i in range(10_000_000):
            result += i
        print("Heavy computation finished")
        return result
    
    print("Wrong way (blocks the event loop):")
    start = time.time()
    result = heavy_computation()  # This blocks the event loop!
    print(f"Time taken: {time.time() - start:.2f}s")
    
    # Correct: Use run_in_executor for CPU-bound tasks
    print("\nCorrect way (using executor):")
    start = time.time()
    loop = asyncio.get_event_loop()
    result = await loop.run_in_executor(None, heavy_computation)
    print(f"Time taken: {time.time() - start:.2f}s")

# Pitfall 3: Not handling exceptions properly in tasks
async def pitfall_unhandled_exceptions():
    print("\nPitfall 3: Not handling exceptions in tasks")
    
    async def will_fail():
        await asyncio.sleep(0.5)
        raise ValueError("Task failed!")
    
    # Wrong: Exceptions in tasks can be lost if not handled
    print("Wrong way (exception will be lost):")
    task = asyncio.create_task(will_fail())
    await asyncio.sleep(1)  # Wait long enough for the task to fail
    
    # Correct: Always await tasks to propagate exceptions
    print("\nCorrect way (catching the exception):")
    task = asyncio.create_task(will_fail())
    try:
        await task
    except ValueError as e:
        print(f"Caught exception: {e}")

# Let's show these pitfalls
async def show_pitfalls():
    await pitfall_not_awaiting()
    await pitfall_blocking_loop()
    await pitfall_unhandled_exceptions()

await show_pitfalls()

## Conclusion

Congratulations! You've completed this comprehensive tutorial on Python's asyncio library. Let's recap what we've learned:

1. **Asynchronous Programming Basics**
   - Understanding the difference between synchronous and asynchronous code
   - When to use asyncio (I/O-bound operations)

2. **Core Asyncio Concepts**
   - Coroutines and the `async`/`await` syntax
   - Event loops
   - Tasks and Futures

3. **Advanced Features**
   - Working with queues
   - Synchronization primitives
   - Error handling
   - Timeouts

4. **Practical Applications**
   - Asynchronous HTTP requests
   - File I/O operations
   - Building a simple server
   - Creating a web crawler

5. **Best Practices and Common Patterns**
   - Error handling strategies
   - Debugging techniques
   - Common patterns (scatter-gather, producer-consumer, throttling)
   - Pitfalls to avoid

Remember that asyncio is particularly powerful for I/O-bound applications but isn't suitable for CPU-bound tasks (use `multiprocessing` for those). With the knowledge gained from this tutorial, you should now be able to write efficient, concurrent code using Python's asyncio library.

### Further Resources

To continue learning about asyncio:

1. [Official asyncio documentation](https://docs.python.org/3/library/asyncio.html)
2. [PEP 3156 – Asynchronous IO Support](https://peps.python.org/pep-3156/)
3. [Real Python's asyncio tutorials](https://realpython.com/async-io-python/)
4. Popular asyncio libraries:
   - [aiohttp](https://docs.aiohttp.org/) – HTTP client/server
   - [FastAPI](https://fastapi.tiangolo.com/) – Web framework built on asyncio
   - [asyncpg](https://magicstack.github.io/asyncpg/) – PostgreSQL client
   - [aiomysql](https://aiomysql.readthedocs.io/) – MySQL client
   - [motor](https://motor.readthedocs.io/) – MongoDB client